# Quick Start Tutorial: Model Training

## Learning Objectives

In this tutorial you will learn:
1. How to design an observation set for your use case
2. How to materialize training data
3. How your ML training environment can consume training data

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn the about catalogs
* activate a pre-built catalog

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

20:59:40 | INFO     | Using configuration file at: /Users/jevonyeoh/.featurebyte/config.yaml
20:59:40 | WARNING  | No valid profile specified. Update config file or specify valid profile name with "use_profile".
20:59:40 | INFO     | (1/4) Starting featurebyte services
Container redis  Running
Container spark-thrift  Running
Container mongo-rs  Running
Container featurebyte-server  Running
Container featurebyte-worker  Running
Container mongo-rs  Waiting
Container mongo-rs  Waiting
Container redis  Waiting
Container mongo-rs  Healthy
Container mongo-rs  Healthy
Container redis  Healthy
20:59:41 | INFO     | (2/4) Creating local spark feature store
20:59:41 | INFO     | (3/4) Import datasets
20:59:56 | INFO     | Dataset grocery already exists, skipping import
20:59:56 | INFO     | Dataset healthcare already exists, skipping import
20:59:56 | INFO     | Dataset creditcard already exists, skipping import
20:59:56 | INFO     | (4/4) Playground environment started successfully. Ready to go

### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.QuickStartModelTraining)

Cleaning up existing tutorial catalogs


20:59:56 | INFO     | Catalog activated: quick start model training 20230802:2053


Cleaning catalog: quick start model training 20230802:2053
  2 observation tables
Done! |████████████████████████████████████████| 100% in 12.3s (0.08%/s)        
Done! |████████████████████████████████████████| 100% in 12.3s (0.08%/s)        


21:00:23 | INFO     | Catalog activated: quick start model training 20230802:2100


Building a quick start catalog for model training named [quick start model training 20230802:2100]
Creating new catalog
Catalog created
Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.1s
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.

### Example: Create views from tables in the Catalog

In [3]:
# create the views
grocery_customer_view = catalog.get_view("GROCERYCUSTOMER")
grocery_invoice_view = catalog.get_view("GROCERYINVOICE")
grocery_items_view = catalog.get_view("INVOICEITEMS")
grocery_product_view = catalog.get_view("GROCERYPRODUCT")

## Create an observation set for your use case

Learning Objectives

In this section you will learn:
* the purpose of observation sets
* the relationship between entities, point in time, and observation sets
* how to design an observation set suitable for training data

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

### Concept: Materialization

A feature in FeatureByte is defined by the logical plan for its computation. The act of computing the feature is known as Feature Materialization.

The materialization of features is made on demand to fulfill historical requests, whereas for prediction purposes, feature values are generated through a batch process called a "Feature Job". The Feature Job is scheduled based on the defined settings associated with each feature.

### Concept: Observation set

An observation set combines entity key values and historical points-in-time, for which you wish to materialize feature values.

The observation set can be a pandas DataFrame or an ObservationTable object representing an observation set in the feature store. An accepted serving name must be used for the column containing the entity values. The column containing points-in-time must be labelled "POINT-IN-TIME" and the point-in-time timestamps should be in UTC.

### Concept: Point in time

A point-in-time for a feature refers to a specific moment in the past with which the feature's values are associated.

It is a crucial aspect of historical feature serving, which allows machine learning models to make predictions based on historical data. By providing a point-in-time, a feature can be used to train and test models on past data, enabling them to make accurate predictions for similar situations in the future.

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

In [4]:
# get the feature list for the target
customer_target = catalog.get_target("next_customer_sales_14d")

# display details about the target
display(customer_target.info())

{
  'id': '64ca53c9d4aa2740d2dab6a8',
  'target_name': 'next_customer_sales_14d',
  'entities': [
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'quick start model training 20230802:2100'
    }
  ],
  'window': '14d',
  'has_recipe': True,
  'created_at': '2023-08-02T13:02:01.694000',
  'updated_at': None,
  'primary_table': [
    {
      'name': 'GROCERYINVOICE',
      'status': 'PUBLIC_DRAFT',
      'catalog_name': 'quick start model training 20230802:2100'
    }
  ],
  'metadata': {
    'input_columns': {
      'Input0': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Timestamp',
        'semantic': 'event_timestamp'
      },
      'Input1': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Amount',
        'semantic': None
      }
    },
    'derived_columns': {},
    'aggregations': {
      'F0': {
        'name': 'next_customer_sales_14d',
        'column': 'Input1',
        'function': 'sum',
        'keys': [
          'GroceryCustomerGuid'
        ],
        'window': '14d',
        'category': None,
        'filter': False
      }
    },
    'post_aggregation': {
      'name': 'next_customer_sales_14d',
      'inputs': [
        'F0'
      ],
      'transforms': [
        'is_null',
        'conditional'
      ]
    }
  },
  'namespace_description': None,
  'description': None
}

In [5]:
# create a large observation table from a view

# filter the view to exclude points in time that won't have data for historical windows
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-04-01")) & (
    grocery_invoice_view["Timestamp"] < pd.to_datetime("2023-04-01")
)
observation_set_view = grocery_invoice_view[filter].copy()

# create a new observation table
observation_table = observation_set_view.create_observation_table(
    name="10,000 Customers immediately after each purchase from May-22 to Mar-23",
    sample_rows=10000,
    columns=["Timestamp", "GroceryCustomerGuid"],
    columns_rename_mapping={
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    },
)

# if the observation table isn't too large, you can materialize it
display(observation_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 21.5s (0.05%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-04-05 06:51:50,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-04-05 18:55:03,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-04-08 13:10:00,5c96089d-95f7-4a12-ab13-e082836253f1
3,2022-04-11 11:49:05,5c96089d-95f7-4a12-ab13-e082836253f1
4,2022-05-20 13:03:26,5c96089d-95f7-4a12-ab13-e082836253f1
...,...,...
9995,2022-05-18 15:42:43,7a024068-3f99-4114-9d90-3a61f679be51
9996,2022-05-23 11:28:53,7a024068-3f99-4114-9d90-3a61f679be51
9997,2022-06-02 17:16:51,7a024068-3f99-4114-9d90-3a61f679be51
9998,2022-06-05 14:08:37,7a024068-3f99-4114-9d90-3a61f679be51


## Materialize Training Data

Learning Objectives

In this section you will learn:
* how to create a target observation table
* how to create historical training data using the target observation table

### Example: Get target values


In [6]:
# Materialize the target
training_data_target_table = customer_target.compute_target_table(
    observation_table, observation_table_name="target_observation_table"
)

display(training_data_target_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 25.1s (0.04%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d
0,2022-11-14 14:07:14,abdef773-ab72-43b6-8e77-050804c1c5fc,111.22
1,2022-11-05 19:48:29,776ed61f-ae99-40b4-989b-1195e4901090,15.42
2,2022-11-10 17:59:38,6bb41075-390e-43ba-b416-55d802ede820,35.34
3,2022-07-12 16:13:42,cc4220ec-16ab-4bb9-991d-deef994bf27a,115.54
4,2022-11-25 16:15:36,7bce4665-c8f7-4bc3-94f1-df88fb236ee8,166.49
...,...,...,...
9995,2022-10-23 09:37:54,df23dfe3-fede-46aa-b283-422458c11767,37.35
9996,2022-06-20 06:07:19,f56f0029-0cf3-4c28-bbec-2a8292722a32,4.07
9997,2022-08-15 15:46:59,f6a783f7-5091-46fa-8ebf-aa13ec868234,136.39
9998,2022-08-16 14:34:40,f6a783f7-5091-46fa-8ebf-aa13ec868234,136.88


### Example: Get historical values with target

In [7]:
# list the feature lists
display(catalog.list_feature_lists())

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64ca53c2d4aa2740d2dab6a4,Features,8,DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-08-02T13:01:55.266000


In [8]:
# get the feature list
feature_list = catalog.get_feature_list("Features")

Loading Feature(s) |████████████████████████████████████████| 8/8 [100%] in 0.1s


In [ ]:
# Compute the historical feature table by passing in the observation table that contains the target values
training_table_features = feature_list.compute_historical_feature_table(
    training_data_target_table,
    historical_feature_table_name="customer training table - invoices Apr-22 to Mar-23 - features only",
)

# display the training data
training_data = training_table_features.to_pandas()
display(training_data)

Working... |████                                    | ▆▄▂ 10% in 32s (~4:39, 0.0

## Consuming training data

Learning Objectives

In this section you will learn:
* how to save a training file
* how to use a pandas data frame

### Example: Save the training data to a file

In [ ]:
# save training data as a csv file
training_data.to_csv("training_data.csv", index=False)

In [ ]:
# save the training file as a parquet file
training_data.to_parquet("training_data.parquet")

### Example: Training a scikit learn model

Note that you will need to install scikit learn https://scikit-learn.org/stable/install.html

In [ ]:
# EDA on the training data
training_data.describe()

In [ ]:
# do any columns in the training data contain missing values?
training_data.isna().any()

In [ ]:
! pip install scikit-learn

In [ ]:
# use sklearn to train a random forest regression model on the training data
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    training_data.drop(columns=["GROCERYCUSTOMERGUID", "POINT_IN_TIME"]),
    training_data["next_customer_sales_14d"],
    test_size=0.2,
    random_state=42,
)

# train the model
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train)

# get predictions
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)

# save the model
import joblib

joblib.dump(model, "model.pkl")

## Next Steps

Now that you've completed the quick-start feature engineering tutorial, you can put your knowledge into practice or learn more:<br>
1. Learn more about materializing features via the "Deep Dive Materializing Features" tutorial
2. Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" workspaces
3. Learn more about feature governance via the "Quick Start Feature Governance" tutorial